In [1]:
import pandas as pd
import glob
import os
import gutenberg
from gutenberg.cleanup import strip_headers
import sqlite3 as sql
import requests
import urllib
import json
from collections import Counter
from ast import literal_eval

In [2]:
conn = sql.connect('pg-pr-eng.db')
c = conn.cursor()

In [3]:
df = pd.read_sql('select `id`, `title_prEng`, `author`, `wikipedia` from pg', conn)

DatabaseError: Execution failed on sql 'select `id`, `title_prEng`, `author`, `wikipedia` from pg': no such table: pg

In [127]:
class DBPedia(): 
    """ Looks up metadata about a book from DBPedia, 
    using fuzzy matching on the title and author. """
    def __init__(self, title=None, author=None, resource=None):
        if resource==None and (title==None or author==None): 
            print('Must specify either the resource URI stub, or title and author.')
            raise
        if resource is not None: 
            resource = urllib.parse.unquote(resource)
            self.resourceURI = "http://dbpedia.org/resource/" + resource
        else: 
            response = self.getMeta(title, author)
            parsed = self.parseMeta(response)
            self.resourceURI = self.getResourceURI(parsed)
    
        if self.resourceURI is not None: 
            resourceResponse = self.queryDBP(self.resourceURI)
            resourceParsed = self.parseMeta(resourceResponse)
            self.info = self.getInfo(resourceParsed)
            if self.info is not None: 
                self.pubDate = self.getPubDate()
                self.subjects = self.getSubjects()
                self.literaryGenres = self.getLiteraryGenres()
            else: 
                self._abort()
        else: 
            self._abort()
        
    def _abort(self): 
        self.info = None 
        self.pubDate = None
        self.subjects = None
        self.literaryGenres = None
    
    def getMeta(self, title, author): 
        query = """select distinct ?book
                where {
                  ?book a dbo:Book .
                  ?book dbo:author ?author .
                  ?book dbp:name ?name .
                  FILTER (regex(?name, "%s"))
                  FILTER (regex(?author, "%s"))
                } 
                LIMIT 100""" % (title, author)
        response = requests.get("http://dbpedia.org/sparql", 
                                params=[('query', query), 
                                        ('format', 'application/json')])
        return response

    def parseMeta(self, dbpResponse): 
        try: 
            parsed = json.loads(dbpResponse.text)
        except: 
            print("Couldn't decode JSON: ", dbpResponse.text)
            return None
        return parsed

    def getResourceURI(self, jsonMeta): 
        try: 
            bindings = jsonMeta['results']['bindings']
        except KeyError: 
            print("Couldn't find the appropriate results in the JSON: ", jsonMeta)
            return None
        try: 
            uri = bindings[0]['book']['value']
        except IndexError: 
            print("Couldn't find a resource URI. There is no 0th element in the JSON: ", jsonMeta)
            return None
        return uri
    
    def queryDBP(self, resourceURI):
        query = 'DESCRIBE <%s>' % resourceURI
        params = [('default-graph-uri', 'http://dbpedia.org'), 
                  ('query', query), ('format', 'application/json')]
        r = requests.get("http://dbpedia.org/sparql", params=params)
        return r

    def getInfo(self, jsonData):
        try: 
            data = jsonData[self.resourceURI]
            return data
        except KeyError: 
            print('Strangely, there is no entry in items for this item. Jsondata: ', jsonData)
        
    
    def getPubDate(self): 
        try: 
            releaseDate = self.info['http://dbpedia.org/ontology/releaseDate']
        except KeyError: 
            print('No releaseDate in self.info ontology. Trying property.')
            print('Available info: ', self.info.keys())
            try: 
                releaseDate = self.info['http://dbpedia.org/property/releaseDate']
            except KeyError: 
                print('No releaseDate in self.info property. Giving up.')
                return None
        try: 
            firstRelease = releaseDate[0]
        except IndexError: 
            print('There is no 0th release, apparently: ', releaseDate)
            return None
        try: 
            date = firstRelease['value']
        except KeyError: 
            print('Strangely, no actual date in releaseDate:', releaseDate)
            return None
        return date
    
    def getSubjects(self): 
        subjects = []
        try: 
            subjs = self.info['http://purl.org/dc/terms/subject']
        except KeyError: 
            print('No subject in self.info.')
            return None
        for subj in subjs: 
            subject = subj['value'].split('/')[-1]
            if subject.startswith('Category:'): 
                subject = subject[9:]
            subjects.append(subject)
        return subjects
    
    def getLiteraryGenres(self): 
        literaryGenres = []
        try: 
            genres = self.info['http://dbpedia.org/ontology/literaryGenre']
        except KeyError: 
            print('No literary genre in self.info.')
            return None
        for genre in genres: 
            literaryGenre = genre['value'].split('/')[-1]
            literaryGenres.append(literaryGenre)
        return literaryGenres

In [ ]:
uly = DBPedia('Ulysses', 'Joyce')

In [21]:
pp = DBPedia("Pilgrim's Progress", "Bunyan")

No releaseDate in self.info ontology. Trying property.
Available info:  dict_keys(['http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'http://www.w3.org/2000/01/rdf-schema#label', 'http://www.w3.org/2000/01/rdf-schema#comment', 'http://www.w3.org/2002/07/owl#sameAs', 'http://www.w3.org/ns/prov#wasDerivedFrom', 'http://dbpedia.org/ontology/abstract', 'http://dbpedia.org/ontology/author', 'http://xmlns.com/foaf/0.1/name', 'http://dbpedia.org/property/name', 'http://xmlns.com/foaf/0.1/depiction', 'http://xmlns.com/foaf/0.1/isPrimaryTopicOf', 'http://dbpedia.org/ontology/thumbnail', 'http://purl.org/dc/terms/subject', 'http://dbpedia.org/ontology/wikiPageID', 'http://dbpedia.org/ontology/wikiPageRevisionID', 'http://dbpedia.org/ontology/wikiPageExternalLink', 'http://dbpedia.org/property/author'])
No releaseDate in self.info property. Giving up.


In [52]:
df

,id,title_prEng,author,wikipedia,releaseDate,subjects
0,100,The Complete Works of William Shakespeare,"Shakespeare, William",None,NaN,NaN
1,10002,The House on the Borderland,"Hodgson, William Hope",['https://fr.wikipedia.org/wiki/La_Maison_au_b...,NaN,NaN
2,10007,Carmilla,"Le Fanu, Joseph Sheridan","['https://fr.wikipedia.org/wiki/Carmilla', 'ht...",NaN,NaN
3,10021,Tenterhooks,"Leverson, Ada",None,NaN,NaN
4,10029,The Hunt Ball Mystery,"Magnay, William, Sir",None,NaN,NaN
5,10039,"The Works of Aphra Behn, Volume III","Behn, Aphra",None,NaN,NaN
6,10046,Salute to Adventurers,"Buchan, John",None,NaN,NaN
7,10049,Old Lady Mary: A Story of the Seen and the Unseen,"Oliphant, Mrs. (Margaret)",None,NaN,NaN
8,10050,A Little Pilgrim: Stories of the Seen and the ...,"Oliphant, Mrs. (Margaret)",None,NaN,NaN
9,10051,The Little Pilgrim: Further Experiences.: Stor...,"Oliphant, Mrs. (Margaret)",None,NaN,NaN


In [ ]:
twi = DBPedia(resource="The_Twilight_of_the_Gods_and_Other_Tales")

In [8]:
def getResource(wikiPagesRaw): 
    try: 
        wikiList = literal_eval(wikiPagesRaw)
    except ValueError: 
        return None
    for item in wikiList: 
        if 'en.wikipedia.org' in item: 
            return item.split('/')[-1]
    return None

In [22]:
def sanitizeTitle(title): 
    # Grab the first part of the title, before any colon. 
    if ':' in title: 
        title = title.split(':')[0]
    # If there are quotation marks, remove everything after. 
    if '"' in title: 
        title = title.split('"')[0]
    return title

In [134]:
def lookupStuff(start=0):
    times = 0
    for i, row in df.iterrows(): 
        if i<start: 
            continue
        print('------------ Processing row %s -------------' % i)
        resource = getResource(row.wikipedia)
        times += 1
        if resource is not None: 
            print('Getting resource: ', resource)
            book = DBPedia(resource=resource)
        else: 
            print('Looking up by title and author.')
            title = sanitizeTitle(row.title_prEng)
            print('Using title: ', title)
            author = row.author
            if author is not None: 
                author = row.author.split(',')[0]
                print('Using author: ', author)
                book = DBPedia(title = title, author=author)
        if book is not None: 
            pubdate = book.pubDate
            print('Pubdate: ', pubdate)
            df.set_value(i, 'releaseDate', pubdate)
            subjects = book.subjects
            print('Subjects: ', subjects)
            if subjects is not None: 
                df.set_value(i, 'subjects', str(subjects))
            literaryGenres = book.literaryGenres
            print('Literary genres: ', literaryGenres)
            if literaryGenres is not None: 
                df.set_value(i, 'literaryGenres', str(literaryGenres))
            if book.info is not None: 
                df.set_value(i, 'info', str(book.info))

In [146]:
lookupStuff(start=6330)

------------ Processing row 6330 -------------
Looking up by title and author.
Using title:  Joan of the Sword Hand
Using author:  Crockett
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Pubdate:  None
Subjects:  None
Literary genres:  None
------------ Processing row 6331 -------------
Looking up by title and author.
Using title:  A Legend of Reading Abbey
Using author:  MacFarlane
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}
Pubdate:  None
Subjects:  None
Literary genres:  None
------------ Processing row 6332 -------------
Looking up by title and author.
Using title:  Byron
Using author:  Edgcumbe
Couldn't find a resource URI. There is no 0th element in the JSON:  {'head': {'link': [], 'vars': ['book']}, 'results': {'distinct': Fal

In [147]:
allSubjs = []
for i, row in df.iterrows(): 
    subjs = row['subjects']
    try: 
        subjs = literal_eval(subjs)
    except: 
        continue
    for subj in subjs: 
        allSubjs.append(subj)
        

In [148]:
Counter(allSubjs).most_common(50)

[('Novels_first_published_in_serial_form', 127),
 ('British_novels_adapted_into_films', 99),
 ('Victorian_novels', 95),
 ('British_novels', 82),
 ('English_novels', 81),
 ('Novels_adapted_into_plays', 52),
 ('Novels_set_in_London', 41),
 ('Historical_novels', 40),
 ('Chapman_&_Hall_books', 38),
 ('Gothic_novels', 38),
 ('Novels_by_H._Rider_Haggard', 37),
 ('19th-century_novels', 35),
 ('Novels_by_Anthony_Trollope', 30),
 ('Debut_novels', 28),
 ('Novels_by_H._G._Wells', 27),
 ('Novels_by_Charles_Dickens', 27),
 ('Picaresque_novels', 24),
 ('Novels_by_P._G._Wodehouse', 22),
 ('Novels_by_Walter_Scott', 22),
 ('British_science_fiction_novels', 21),
 ('British_fantasy_novels', 21),
 ('Macmillan_Publishers_books', 21),
 ('1890_novels', 21),
 ('Novels_set_in_England', 20),
 ('Novels_adapted_into_comics', 19),
 ('1890s_fantasy_novels', 19),
 ('Novels_by_Edward_Bulwer-Lytton', 19),
 ("British_children's_novels", 17),
 ('Scottish_novels', 16),
 ('Literature_featuring_anthropomorphic_characters',

In [160]:
df[df['info'].notnull()]

,id,title_prEng,author,wikipedia,releaseDate,wikiSubjects,literaryGenres,info
1,10002,The House on the Borderland,"Hodgson, William Hope",['https://fr.wikipedia.org/wiki/La_Maison_au_b...,None,"['Novels_by_William_Hope_Hodgson', 'Science_fi...",['Horror_fiction'],{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
2,10007,Carmilla,"Le Fanu, Joseph Sheridan","['https://fr.wikipedia.org/wiki/Carmilla', 'ht...",None,"['1872_novels', 'Lesbian_novels', 'Fictional_v...",['Gothic_fiction'],{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
16,10095,"The Twilight of the Gods, and Other Tales","Garnett, Richard",['https://en.wikipedia.org/wiki/The_Twilight_o...,None,"['Fantasy_short_story_collections', '1888_shor...",['Fantasy'],{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
24,1013,The First Men in the Moon,"Wells, H. G. (Herbert George)",['https://fi.wikipedia.org/wiki/Ensimm%C3%A4is...,None,"['Moon_in_fiction', 'British_novels_adapted_in...",NaN,{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
27,10148,The Merry Adventures of Robin Hood,"Pyle, Howard",None,None,"[""American_children's_novels"", 'Novels_by_Howa...","['Historical_fiction', ""Children's_literature""]",{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
28,10150,Dracula's Guest,"Stoker, Bram",None,None,"['Dracula_in_written_fiction', 'Books_publishe...",NaN,{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
29,10161,English Poets of the Eighteenth Century,None,None,None,"['Dracula_in_written_fiction', 'Books_publishe...",NaN,{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
30,10165,Across the Zodiac: The Story of a Wrecked Record,"Greg, Percy",['https://en.wikipedia.org/wiki/Across_the_Zod...,None,"['Mars_in_fiction', 'Novels_set_in_the_1830s',...",['List_of_science_fiction_novels'],{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...
31,1019,"Poems by Currer, Ellis, and Acton Bell","Brontë, Charlotte",['https://fi.wikipedia.org/wiki/Poems_by_Curre...,None,"['1846_books', 'Brontë_poems', 'English_poetry...",NaN,{'http://www.w3.org/2000/01/rdf-schema#label':...
34,1024,The Wrecker,"Osbourne, Lloyd",['https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D...,None,"['Scottish_novels', 'Midway_Atoll', '1892_nove...",['Adventure_fiction'],{'http://www.w3.org/1999/02/22-rdf-syntax-ns#t...


In [158]:
df = df.rename(columns={"subjects": "wikiSubjects"})

In [159]:
df.to_sql('pg', conn, if_exists='append')